In [7]:
# python uses its default encoder to encode objects 
#into Json objects

|Python |JSON  |
|:----|:----|
| `dict` | object `{...}`|
| `list`, `tuple` | array `[...]` |
| `str`  | string `"..."`|
| `int`, `float` | number |
| `int` or `float` `Enums` | number |
| `bool` | `true` or `false` |
| `None` | `null` |

In [1]:
import json

In [2]:
default_encoder = json.JSONEncoder()

In [3]:
default_encoder.encode([1,2,3])

'[1, 2, 3]'

In [5]:
default_encoder.encode( 1+ 1j)

TypeError: Object of type complex is not JSON serializable

We can actually extend this `JSONEncoder` class and override 
the `default` method. We can then add in support for whatever 
type we want to use, and pass any other types to the parent class to handle (either serialize the data or raise a `TypeError` exception). 

In [13]:
dir(default_encoder)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'allow_nan',
 'check_circular',
 'default',
 'encode',
 'ensure_ascii',
 'indent',
 'item_separator',
 'iterencode',
 'key_separator',
 'skipkeys',
 'sort_keys']

In [11]:
dir(json.JSONEncoder)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'default',
 'encode',
 'item_separator',
 'iterencode',
 'key_separator']

In [14]:
import json
from datetime import datetime

class CustomJSONEncoder(json.JSONEncoder):
    
    # overriding the default method of jsonencoder
    
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            super().default(arg)
            

In [16]:
d = {'name' :'test', 'time': datetime.utcnow() }

In [17]:
json.dumps(d, cls = CustomJSONEncoder  )

'{"name": "test", "time": "2019-12-21T19:57:05.363208"}'

In [21]:
# or for returning more stuff for a new obj than just usual.

In [24]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, datetime):
            obj = dict(
                datatype="datetime",
                iso=arg.isoformat(),
                date=arg.date().isoformat(),
                time=arg.time().isoformat(),
                year=arg.year,
                month=arg.month,
                day=arg.day,
                hour=arg.hour,
                minutes=arg.minute,
                seconds=arg.second
            )
            return obj
        else:
            return super().default(arg)

In [25]:
json.dumps(d, cls = CustomJSONEncoder  )

'{"name": "test", "time": {"datatype": "datetime", "iso": "2019-12-21T19:57:05.363208", "date": "2019-12-21", "time": "19:57:05.363208", "year": 2019, "month": 12, "day": 21, "hour": 19, "minutes": 57, "seconds": 5}}'

In [26]:
print(json.dumps(d, cls = CustomJSONEncoder, indent  = 2))

{
  "name": "test",
  "time": {
    "datatype": "datetime",
    "iso": "2019-12-21T19:57:05.363208",
    "date": "2019-12-21",
    "time": "19:57:05.363208",
    "year": 2019,
    "month": 12,
    "day": 21,
    "hour": 19,
    "minutes": 57,
    "seconds": 5
  }
}


In [18]:
# but we can't override usual types, like str

In [20]:
# so its only for new type of objects not in the table above.

In [27]:
def custom_encoder(arg):
    print('Custom encoder called...')
    if isinstance(arg, str):
        return f'some string: {arg}'

In [28]:
json.dumps({'name': 'Python'}, default=custom_encoder)

'{"name": "Python"}'

In [29]:
d = {
    'a': float('inf'),
    'b': [1, 2, 3]
}

In [30]:
d

{'a': inf, 'b': [1, 2, 3]}

In [31]:
json.dumps(d)

'{"a": Infinity, "b": [1, 2, 3]}'

In [33]:
json.dumps(d, allow_nan=False) # to not allow infinity shit

ValueError: Out of range float values are not JSON compliant

In [34]:
d = {10: "int", 10.5: "float", 1+1j: "complex"}

In [35]:
json.dumps(d)

TypeError: keys must be str, int, float, bool or None, not complex

As you can see we get an exception. We may want to simply ignore that exception and not include the offending key/value pair in our serialization:

In [36]:
json.dumps(d, skipkeys=True)

'{"10": "int", "10.5": "float"}'

We can even change how the serialization is rendered (which of course means we may no longer have actual JSON):

In [37]:
d = {
    'name': 'Python',
    'age': 27,
    'created_by': 'Guido van Rossum',
    'list': [1, 2, 3]
}

In [38]:
json.dumps(d)

'{"name": "Python", "age": 27, "created_by": "Guido van Rossum", "list": [1, 2, 3]}'

In [39]:
print(json.dumps(d, indent='---', separators=('', ' = ')))

{
---"name" = "Python"
---"age" = 27
---"created_by" = "Guido van Rossum"
---"list" = [
------1
------2
------3
---]
}


In [40]:
# we can change this directly by overriding the __init__ of 
# jsonencoder

In [41]:
class CustomEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(skipkeys=True, 
                         allow_nan=False, 
                         indent='---', 
                         separators=('', ' = ')
                        )
        
    def default(self, arg):
        if isinstance(arg, datetime):
            return arg.isoformat()
        else:
            return super().default(arg)

In [42]:
d = {
    'time': datetime.utcnow(),
    1+1j: "complex",
    'name': 'Python'
}

In [43]:
print(json.dumps(d, cls=CustomEncoder))

{
---"time" = "2019-12-21T20:08:25.895279"
---"name" = "Python"
}
